In [4]:
from transformers import BertTokenizerFast, BertModel


tokenizer_fast = BertTokenizerFast.from_pretrained('dmis-lab/biobert-base-cased-v1.1', 
                                          do_lower_case=True,
                                          padding = True,
                                          truncation=True,
                                          add_special_tokens = True,
                                          model_max_length = 1000000000)

In [6]:
from transformers import BertForQuestionAnswering
import torch
model = BertForQuestionAnswering.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
checkpoint = torch.load('./train/checkpoint_mnli_squad_factoid_3epochs_seed.pt',map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f

<All keys matched successfully>

In [7]:
test_question = "What is my name?"
test_context = "My name is Marga"

inputs = tokenizer_fast(test_question, 
                   test_context, 
                   truncation=True, 
                   padding=True,
                   max_length=500, 
                   return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0]
text_tokens = tokenizer_fast.convert_ids_to_tokens(input_ids)
outputs = model(**inputs)
answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits
# Get the most likely beginning of answer 
answer_start = torch.argmax(answer_start_scores)  
# Get the most likely end of answer 
answer_end = torch.argmax(answer_end_scores) + 1  
answer = tokenizer_fast.convert_tokens_to_string(tokenizer_fast.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

In [8]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-0.1529, -0.1529, -0.1529, -0.1529, -0.1529, -0.1529, -0.1529, -0.1529,
         -0.1529, -0.1529, -0.1529, -0.1529, -0.1529]],
       grad_fn=<SqueezeBackward1>), end_logits=tensor([[0.3687, 0.3687, 0.3687, 0.3687, 0.3687, 0.3687, 0.3687, 0.3687, 0.3687,
         0.3687, 0.3687, 0.3687, 0.3687]], grad_fn=<SqueezeBackward1>), hidden_states=None, attentions=None)

In [9]:
from torch.jit import trace

In [21]:
traced = trace(model,
               [inputs['input_ids'],inputs['attention_mask'], inputs['token_type_ids']],
               strict = False)

In [2]:
import dill
dill.load_session('globalsave_mnli_squad.pkl',ignore = True)

AttributeError: Can't get attribute 'BertForMultiLabelSequenceClassification' on <module '__main__'>

In [1]:
2+2

4